In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import requests
from io import BytesIO
from PIL import Image
import ast

In [30]:
annotations_results = pd.read_csv("data/20200606/Batch_4061620_batch_results.csv")

color = {"car":"#cd3232", "truck": "#32cd32", "bus": "#3c32cd", "motercycle": "#cd32c8", "cycle":"#74d3d6"}

batch = "Batch_4061620_batch_results"

In [31]:
len(annotations_results)

2119

In [32]:
for i, each in annotations_results.iterrows():
    if each["AssignmentStatus"] == "Submitted":
        image_url = requests.get(each["Input.image_url"])
        
        img = Image.open(BytesIO(image_url.content))
        im = np.array(img, dtype=np.uint8)

        hit_id = each["HITId"]
        work_time = each["WorkTimeInSeconds"]
        approval_rate = each["LifetimeApprovalRate"].replace("/", ":")
        file_name = each['Input.image_url'].split('/')[-1]

        worker_answer = each["Answer.annotatedResult.boundingBoxes"]

        fig,ax = plt.subplots(1)
        ax.set_axis_off()
        fig.add_axes(ax)
        ax.imshow(im)

        # Here we are using ast.literal_eval because the results are in string format
        for answer in ast.literal_eval(worker_answer):
            rect = patches.Rectangle((answer['left'],answer['top']),answer['width'],answer['height'],linewidth=1,edgecolor=color[answer["label"]],facecolor='none')
            ax.text(answer['left'],answer['top'], answer["label"], fontsize=3)
            ax.add_patch(rect)
        # Save the bounding box with images
        plt.savefig(f"output/{batch}/{work_time}@{approval_rate}@{file_name[:-4]}.png", dpi = 300, pad_inches=0.0, bbox_inches='tight')    
        plt.close()

In [22]:
annotation_results_corrected = annotations_results[annotations_results['AssignmentStatus'] == "Submitted"]

In [28]:
len(annotation_results_corrected)

NameError: name 'annotation_results' is not defined

In [24]:
output_name = batch.split('/')[0] + '_corrected'

In [25]:
annotation_results_corrected.to_csv(f'{output_name}.csv', index= False)